<a href="https://colab.research.google.com/github/emily2925/DS-Class-Project-Titanic/blob/main/%E9%90%B5%E9%81%94%E5%B0%BC%E8%99%9F%E5%B0%88%E6%A1%88%E5%AF%A6%E4%BD%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

目的：找出鐵達尼號事件生存的關鍵因素

Packages

In [49]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Data Reading and Explore
1. 整體而言，死:生=5:3
2. class-> 1:2:3 = 2:1:4，多半住3等艙
3. sex->男:女 = 5:3
4. 多半在S港上船
5. 多半親屬朋友家人數為1

In [2]:
# data definition:https://www.kaggle.com/c/titanic/data
df = pd.read_csv('https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/train.csv')

In [27]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [80]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [85]:
for i in (["Survived", "Pclass", "Sex", "SibSp", "Parch", "Embarked"]):
  print(df[[i, "PassengerId"]].groupby([i]).count())

          PassengerId
Survived             
0                 549
1                 342
        PassengerId
Pclass             
1               216
2               184
3               491
        PassengerId
Sex                
female          314
male            577
       PassengerId
SibSp             
0              608
1              209
2               28
3               16
4               18
5                5
8                7
       PassengerId
Parch             
0              678
1              118
2               80
3                5
4                4
5                5
6                1
          PassengerId
Embarked             
C                 168
Q                  77
S                 644


In [33]:
df_all = df
df_target = df[["Survived"]]
df_continuous = df[["Survived", "Age", "SibSp", "Parch", "Fare"]]
df_label = df[["Survived", "PassengerId", "Pclass", "Name", "Sex", "Ticket", "Cabin", "Embarked", "Age"]]

In [13]:
print("-----Data Basic Info------")
print("1. all data")
print(df.info())
print("\n")

print("2.df_continuous data")
print(df_continuous.info())
print("\n")

print("3.label data")
print(df_label.info())
print("\n")

print("4.target data")
print(df_target.info())

-----Data Basic Info------
1. all data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


2.df_continuous data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  ---

Data Reading and Explore - Numeric Data
1. Fare 關聯最大，正向
2. Age負向、SibSp負向、Parch正向
3. 從數字分配來看，其餘差異不大，明顯Live組Fare比較高

In [31]:
df_continuous.groupby("Survived").count()

,Age,SibSp,Parch,Fare
Survived,,,,
0,424,549,549,549
1,290,342,342,342


In [32]:
print(df_continuous.corr().loc[["Survived"]])

          Survived       Age     SibSp     Parch      Fare
Survived       1.0 -0.077221 -0.035322  0.081629  0.257307


In [40]:
c.head()

,Survived,Age,SibSp,Parch,Fare
0,0,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,1,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,0,35.0,0,0,8.0500


In [79]:
fig = plt.figure(figsize=(6, 16))
col = ["Age", "SibSp", "Parch", "Fare"]
for i in range(len(col)):
  print(f"--------{i+1}.{col[i]} distribution--------")
  print("Dead Distribution")
  print(df[df["Survived"]==0][col[i]].describe())
  print("Live Distribution")
  print(df[df["Survived"]==1][col[i]].describe())
  print("\n")

# x = 1
# for i in range(len(col)):
#   df_continuous_ = df_continuous[["Survived", col[i]]].groupby("Survived").mean().reset_index()
#   df_continuous__ = df_continuous[["Survived", col[i]]].groupby("Survived").median().reset_index()
#   print(df_continuous_.head())
#   print(df_continuous__.head())
  # plt.subplot(4, 1, x)
  # plt.bar(x=df_continuous_["Survived"], height=df_continuous_[col[i]])
  # plt.xticks([0, 1], [0, 1])
  # x+=1

--------1.Age distribution--------
Dead Distribution
count    424.000000
mean      30.626179
std       14.172110
min        1.000000
25%       21.000000
50%       28.000000
75%       39.000000
max       74.000000
Name: Age, dtype: float64
Live Distribution
count    290.000000
mean      28.343690
std       14.950952
min        0.420000
25%       19.000000
50%       28.000000
75%       36.000000
max       80.000000
Name: Age, dtype: float64


--------2.SibSp distribution--------
Dead Distribution
count    549.000000
mean       0.553734
std        1.288399
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        8.000000
Name: SibSp, dtype: float64
Live Distribution
count    342.000000
mean       0.473684
std        0.708688
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        4.000000
Name: SibSp, dtype: float64


--------3.Parch distribution--------
Dead Distribution
count    549.000000
mean       0.329690
std      

<Figure size 600x1600 with 0 Axes>

Data Reading and Explore - Object Data